In [ ]:
# librerías auxiliares
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# modulos de sklearn
from sklearn.model_selection import StratifiedKFold, train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight

# metricas de desempeño
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

Importamos los datasets

In [ ]:
data_estimador1 = pd.read_csv("dataset_clasificador_binario.csv")
data_estimador2 = pd.read_csv("dataset_clasificador_multiclase.csv")

# Validación de los mejores modelos de clasificación binario realizando extracción de características

La siguiente tabla muestra la mejor combinación de hiperparametros encontrados en la fase de validación para los modelos.

|                        | Hiperparametros |
|------------------------|-----------------|
| SVC                    | kernel = 'linear', gamma = 0.01, C = 0.001                |
| Gradient Boosting tree | n_estimators=5, min_samples_split=4                |


In [ ]:
def entrenamiento_pca_ext_caracteristicas(estimador, X, Y, n_comp, dataframe):
    """
    Esta función realiza la reducción de la dimensionalidad sobre el conjunto de
    datos de entrenamiento, de acuerdo con las particiones especificadas usando PCA

    Parámetros:
        n_comp, int, Número de componentes para reducción
        n_sets,int, Número de particiones
        X: numpy Array de características
        Y: numpy Array  Vector de etiquetas

    Retorna: 
        El valor medio de errores
        Intervalo de confianza del error
        El  valor medio del tiempo de ejecución
    """  
    idx = 0
    skf = StratifiedKFold(n_splits=5)
    for n in n_comp:
      print("# Componentes", n)
      errores_test_recall = []
      errores_test_auc = []
      for train_index, test_index in skf.split(X, Y):  
          X_train, X_test = X[train_index], X[test_index]
          y_train, y_test = Y[train_index], Y[test_index]

          class_weight = compute_class_weight(class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train )
          

          # balanceamos las muestras de entrenamiento
          nm = RandomUnderSampler(sampling_strategy=0.5)
          X_train, y_train = nm.fit_resample(X_train, y_train)

          pca = PCA(n_components=n)
          X_train_pca = pca.fit_transform(X=X_train)
          X_test_pca = pca.transform(X=X_test)

          y_test_pred = 0
          y_test_pred_prob = 0
          if estimador == 'svc':
            # entrenar el modelo usando las caractieristicas transformadas por PCA
            svc = SVC(kernel = 'linear', gamma = 0.01, C = 0.001, probability=True, class_weight = {0:class_weight[0], 1:class_weight[1]})
            svc.fit(X=X_train_pca, y=y_train)
            # prediccion del modelo
            y_test_pred = svc.predict(X=X_test_pca)
            y_test_pred_prob = svc.predict_proba(X_test_pca)[:, 1]
          elif estimador == 'gbt':
            gbt = GradientBoostingClassifier(n_estimators=5, min_samples_split=4)
            weight = []
            for i in y_train:
              weight.append(class_weight[i])
            gbt.fit(X_train_pca,y_train,weight)
            # prediccion del modelo
            y_test_pred = gbt.predict(X=X_test_pca)
            y_test_pred_prob = gbt.predict_proba(X_test_pca)[:, 1]

          #Evaluamos las predicciones del modelo con los datos de  test
          # calculo del recall
          errores_test_recall.append(recall_score(y_true = y_test, y_pred = y_test_pred))
          # calculo del area bajo la curva roc
          errores_test_auc.append(roc_auc_score(y_true=y_test, y_score=y_test_pred_prob))

      dataframe.loc[idx,'# componentes'] = n
      dataframe.loc[idx,'error de test (recall)'] = np.mean(errores_test_recall)
      dataframe.loc[idx,'error de test (auc)'] = np.mean(errores_test_auc)
      idx= idx +1

In [ ]:
x_estimador1 = data_estimador1.drop(['Target'], axis=1).values
y_estimador1 = data_estimador1['Target'].values

## Validamos la maquina de soporte vectorial junto con el algoritmo PCA buscando cual es el número de componentes optimo para entrenar el modelo

In [ ]:
df_svc = pd.DataFrame()
componentes = [2,3,4,5,6,7,8]
entrenamiento_pca_ext_caracteristicas('svc', x_estimador1, y_estimador1, componentes, df_svc)

# Componentes 2
# Componentes 3
# Componentes 4
# Componentes 5
# Componentes 6
# Componentes 7
# Componentes 8


Resultados

In [ ]:
df_svc

,# componentes,error de test (recall),error de test (auc)
0,2.0,1.0,0.614224
1,3.0,1.0,0.438685
2,4.0,1.0,0.732651
3,5.0,1.0,0.837950
4,6.0,1.0,0.891281
5,7.0,1.0,0.895438
6,8.0,1.0,0.892972


En base a los resultados obtenidos vemos que el mejor rendimiento para una maquina de soporte vectorial lo obtenemos entredando este modelo reduciendo el espacio de dimensión a 7 de las 8 caracteristicas.

### Entrenamos una maquina de vectores de soporte con dimension 7 y medimos su rendimiento

In [ ]:
# particionamos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(x_estimador1, y_estimador1, test_size=0.3, random_state=42, stratify=y_estimador1)
# calculamos el peso de las clases
class_weight = compute_class_weight(class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train )
# balanceamos las muestras
# balanceamos las muestras de entrenamiento
nm = RandomUnderSampler(sampling_strategy=0.5)
X_train, y_train = nm.fit_resample(X_train, y_train)
# aplicamos pca sobre los datos de entrenamiento y prueba
pca = PCA(n_components=7)
X_train_pca = pca.fit_transform(X=X_train)
X_test_pca = pca.transform(X=X_test)
# definimos y entrenamos el modelo
svc = SVC(kernel = 'linear', gamma = 0.01, C = 0.001, probability=True, class_weight = {0:class_weight[0], 1:class_weight[1]})
svc.fit(X=X_train_pca, y=y_train)
# medimos el desempeño del modelo tras haber aplicado extracción de características
y_test_pred = svc.predict(X=X_test_pca)
y_test_pred_prob = svc.predict_proba(X_test_pca)[:, 1]
recall_svc = recall_score(y_true = y_test, y_pred = y_test_pred)
auc_svc = roc_auc_score(y_true=y_test, y_score=y_test_pred_prob)
print("------- SVC entrenado con 7 caracteristicas ---------")
print("Recall: ",recall_svc)
print("Auc roc: ",auc_svc)

------- SVC entrenado con 7 caracteristicas ---------
Recall:  1.0
Auc roc:  0.9006099711250075


## Validamos el gradient boosting tree junto con el algoritmo PCA buscando cual es el número de componentes optimo para entrenar el modelo

In [ ]:
df_tree = pd.DataFrame()
componentes = [2,3,4,5,6,7,8]
entrenamiento_pca_ext_caracteristicas('gbt', x_estimador1, y_estimador1, componentes, df_tree)

# Componentes 2
# Componentes 3
# Componentes 4
# Componentes 5
# Componentes 6
# Componentes 7
# Componentes 8


Resultados

In [ ]:
df_tree

,# componentes,error de test (recall),error de test (auc)
0,2.0,0.900000,0.726792
1,3.0,0.939394,0.822344
2,4.0,0.918182,0.828987
3,5.0,0.893939,0.803144
4,6.0,0.875758,0.773122
5,7.0,0.896970,0.812178
6,8.0,0.893939,0.797271


En base a los resultados obtenidos vemos que el mejor rendimiento para un gradient boosting tree lo obtenemos entredando este modelo reduciendo el espacio de dimensión a 3 de las 8 caracteristicas. Esto a pesar de que anteriormente se vio que la mayor varianza explicada la obtenemos con 7 componentes, aunque viendo los resultados de la tabla anterior el tener 7 componentes arrojan practicamente los mismos resultados

### Entrenamos un gradient boosting tree con dimension 3 y medimos su rendimiento

In [ ]:
# particionamos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(x_estimador1, y_estimador1, test_size=0.3, random_state=42, stratify=y_estimador1)
# calculamos el peso de las clases
class_weight = compute_class_weight(class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train )

# balanceamos las muestras de entrenamiento
nm = RandomUnderSampler(sampling_strategy=0.5)
X_train, y_train = nm.fit_resample(X_train, y_train)
weight_ = []
for i in y_train:
  weight_.append(class_weight[i])
# aplicamos pca sobre los datos de entrenamiento y prueba
pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X=X_train)
X_test_pca = pca.transform(X=X_test)
# definimos y entrenamos el modelo
gbt = GradientBoostingClassifier(n_estimators=5, min_samples_split=4)
gbt.fit(X_train_pca,y_train,weight_)
# medimos el desempeño del modelo tras haber aplicado extracción de características
y_test_pred = gbt.predict(X=X_test_pca)
y_test_pred_prob = gbt.predict_proba(X_test_pca)[:, 1]
recall_gbt = recall_score(y_true = y_test, y_pred = y_test_pred)
auc_gbt = roc_auc_score(y_true=y_test, y_score=y_test_pred_prob)
print("------- Gradient boosting tree entrenado con 3 caracteristicas ---------")
print("Recall: ",recall_gbt)
print("Auc roc: ",auc_gbt)

------- Gradient boosting tree entrenado con 3 caracteristicas ---------
Recall:  1.0
Auc roc:  0.9115419665022153


# Validación de los mejores modelos de clasificación multiclase realizando extracción de características

La siguiente tabla muestra la mejor combinación de hiperparametros encontrados en la fase de validación para los modelos.

|                        | Hiperparametros |
|------------------------|-----------------|
| SVC                    | kernel = 'linear', gamma = 0.01, C =10                |
| Gradient Boosting tree | n_estimators=50, min_samples_split=4                |

In [ ]:
def entrenamiento_pca_ext_caracteristicas_multiclase(estimador, X, Y, n_comp, dataframe):
    """
    Esta función realiza la reducción de la dimensionalidad sobre el conjunto de
    datos de entrenamiento, de acuerdo con las particiones especificadas usando PCA

    Parámetros:
        n_comp, int, Número de componentes para reducción
        n_sets,int, Número de particiones
        X: numpy Array de características
        Y: numpy Array  Vector de etiquetas

    Retorna: 
        El valor medio de errores
        Intervalo de confianza del error
        El  valor medio del tiempo de ejecución
    """  
    idx = 0
    skf = StratifiedKFold(n_splits=5)
    for n in n_comp:
      print("# Componentes", n)
      errores_test_recall = []
      errores_test_auc = []
      for train_index, test_index in skf.split(X, Y):  
          X_train, X_test = X[train_index], X[test_index]
          y_train, y_test = Y[train_index], Y[test_index]

          # balanceamos las muestras de entrenamiento
          smote = SMOTE()
          X_train, y_train = smote.fit_resample(X_train, y_train)

          pca = PCA(n_components=n)
          X_train_pca = pca.fit_transform(X=X_train)
          X_test_pca = pca.transform(X=X_test)

          y_test_pred = 0
          y_test_pred_prob = 0
          if estimador == 'svc':
            # entrenar el modelo usando las caractieristicas transformadas por PCA
            svc = SVC(kernel = 'linear', gamma = 0.01, C = 10, probability=True)
            svc = OneVsRestClassifier(svc)
            svc.fit(X=X_train_pca, y=y_train)
            # prediccion del modelo
            y_test_pred = svc.predict(X=X_test_pca)
            y_test_pred_prob = svc.predict_proba(X_test_pca)
          elif estimador == 'gbt':
            gbt = GradientBoostingClassifier(n_estimators=50, min_samples_split=4)
            gbt = OneVsRestClassifier(gbt)
            gbt.fit(X_train_pca,y_train)
            # prediccion del modelo
            y_test_pred = gbt.predict(X=X_test_pca)
            y_test_pred_prob = gbt.predict_proba(X_test_pca)

          #Evaluamos las predicciones del modelo con los datos de  test
          # calculo del recall
          errores_test_recall.append(recall_score(y_true = y_test, y_pred = y_test_pred, average='weighted'))
          # calculo del area bajo la curva roc
          errores_test_auc.append(roc_auc_score(y_true=y_test, y_score=y_test_pred_prob,multi_class='ovr'))

      dataframe.loc[idx,'# componentes'] = n
      dataframe.loc[idx,'error de test (recall)'] = np.mean(errores_test_recall)
      dataframe.loc[idx,'error de test (auc)'] = np.mean(errores_test_auc)
      idx= idx +1

In [ ]:
x_estimador2 = data_estimador2.drop(['Failure Type'], axis=1).values
y_estimador2 = data_estimador2['Failure Type'].values

## Validamos la maquina de soporte vectorial junto con el algoritmo PCA buscando cual es el número de componentes optimo para entrenar el modelo

In [ ]:
df_svc_ovr = pd.DataFrame()
componentes = [2,3,4,5,6,7,8]
entrenamiento_pca_ext_caracteristicas_multiclase('svc', x_estimador2, y_estimador2, componentes, df_svc_ovr)

# Componentes 2
# Componentes 3
# Componentes 4
# Componentes 5
# Componentes 6
# Componentes 7
# Componentes 8


Resultados

In [ ]:
df_svc_ovr

,# componentes,error de test (recall),error de test (auc)
0,2.0,0.527273,0.728789
1,3.0,0.754545,0.926156
2,4.0,0.784848,0.945916
3,5.0,0.933333,0.988562
4,6.0,0.921212,0.990871
5,7.0,0.909091,0.992023
6,8.0,0.906061,0.989770


En base a los resultados obtenidos vemos que el mejor rendimiento para una maquina de soporte vectorial para el problema multiclase lo obtenemos entredando este modelo reduciendo el espacio de dimensión a 5 o 6 de las 8 dimensiones origninales.

### Entrenamos una maquina de vectores de soporte con dimension 5 y medimos su rendimiento

In [ ]:
# particionamos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(x_estimador2, y_estimador2, test_size=0.3, random_state=42, stratify=y_estimador2)

# balanceamos las muestras de entrenamiento
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)
# aplicamos pca sobre los datos de entrenamiento y prueba
pca = PCA(n_components=5)
X_train_pca = pca.fit_transform(X=X_train)
X_test_pca = pca.transform(X=X_test)
# definimos y entrenamos el modelo
svc = SVC(kernel = 'linear', gamma = 0.01, C = 10, probability=True)
svc = OneVsRestClassifier(svc)
svc.fit(X=X_train_pca, y=y_train)
# medimos el desempeño del modelo tras haber aplicado extracción de características
y_test_pred = svc.predict(X=X_test_pca)
y_test_pred_prob = svc.predict_proba(X_test_pca)
recall_svc = recall_score(y_true = y_test, y_pred = y_test_pred, average='weighted')
auc_svc = roc_auc_score(y_true=y_test, y_score=y_test_pred_prob,multi_class='ovr')
print("------- SVC entrenado con 5 caracteristicas ---------")
print("Recall: ",recall_svc)
print("Auc roc: ",auc_svc)

------- SVC entrenado con 5 caracteristicas ---------
Recall:  0.898989898989899
Auc roc:  0.9788303716705735


## Validamos el gradient boosting tree junto con el algoritmo PCA buscando cual es el número de componentes optimo para entrenar el modelo

In [ ]:
df_tree_ovr = pd.DataFrame()
componentes = [2,3,4,5,6,7,8]
entrenamiento_pca_ext_caracteristicas_multiclase('gbt', x_estimador2, y_estimador2, componentes, df_tree_ovr)

# Componentes 2
# Componentes 3
# Componentes 4
# Componentes 5
# Componentes 6
# Componentes 7
# Componentes 8


resultados


In [ ]:
df_tree_ovr

,# componentes,error de test (recall),error de test (auc)
0,2.0,0.690909,0.883873
1,3.0,0.745455,0.924176
2,4.0,0.827273,0.955072
3,5.0,0.827273,0.965904
4,6.0,0.827273,0.968352
5,7.0,0.845455,0.970813
6,8.0,0.842424,0.976266


En base a los resultados obtenidos vemos que el mejor rendimiento para un gradient boostring tree para el problema multiclase lo obtenemos entredando este modelo reduciendo el espacio de dimensión a 7 o dejando las 8 dimensiones originales

### Entrenamos un gradient boosting tree con dimension 7 y medimos su rendimiento

In [ ]:
# particionamos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(x_estimador2, y_estimador2, test_size=0.3, random_state=42, stratify=y_estimador2)
# balanceamos las muestras de entrenamiento
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)
# aplicamos pca sobre los datos de entrenamiento y prueba
pca = PCA(n_components=7)
X_train_pca = pca.fit_transform(X=X_train)
X_test_pca = pca.transform(X=X_test)
# definimos y entrenamos el modelo
gbt = GradientBoostingClassifier(n_estimators=50, min_samples_split=4)
gbt = OneVsRestClassifier(gbt)
gbt.fit(X_train_pca,y_train)
# medimos el desempeño del modelo tras haber aplicado extracción de características
y_test_pred = gbt.predict(X=X_test_pca)
y_test_pred_prob = gbt.predict_proba(X_test_pca)
recall_gbt = recall_score(y_true = y_test, y_pred = y_test_pred, average='weighted')
auc_gbt = roc_auc_score(y_true=y_test, y_score=y_test_pred_prob, multi_class='ovr')
print("------- Gradient boosting tree entrenado con 7 caracteristicas ---------")
print("Recall: ",recall_gbt)
print("Auc roc: ",auc_gbt)

------- Gradient boosting tree entrenado con 7 caracteristicas ---------
Recall:  0.898989898989899
Auc roc:  0.9701508209512624
